# Code Translator for FHE
We need to transcribe a lot of code to work with Microsoft's SEAL Library. Therefore, we need a parser that reads in code, parses it lexically, and then outputs it in SEAL's format. Here's an example snippet that we have to parse and convert to SEAL's format.

        tmp0 = data_ptr[0] + data_ptr[7];
        tmp7 = data_ptr[0] - data_ptr[7];
        data_ptr[0] = tmp10 + tmp11;
        data_ptr[4] = tmp10 - tmp11;
        z1 = (tmp12 + tmp13) * 0.541196100;
        data_ptr[2] = z1 + tmp13 * 0.765366865;
        data_ptr[6] = z1 + tmp12 * - 1.847759065;
        z1 = tmp4 + tmp7;
        z5 = (z3 + z4) * 1.175875602;
        tmp4 *= 0.298631336;
        z1 *= -0.899976223;
        z3 += z5;
        data_ptr[7] = tmp4 + z1 + z3;


In [2]:
SPECIAL_OPERANDS = ["*=", "-=", "+=", "/="]
"""
Parses the text line
DEST = ARG1 (OP)  ... (OP) ARGN
"""
def parse_line(line):
    return NotImplemented 